In [1]:
from pymilvus import connections

In [2]:
from milvus_db import MilvusDB, MilvusQADB

In [5]:

# 1. Tạo mới collection Q&A
qa_db = MilvusQADB("base_qa")
qa_db.create_qa_collection()

In [9]:
qa_db.import_qa_pairs("faq_data.json")

Đã nhập khẩu 2 cặp câu hỏi-trả lời


In [2]:
from pymilvus import (
    Collection,
    CollectionSchema,
    DataType,
    FieldSchema,
    connections,
    utility,
    MilvusException,
)
from typing import List, Dict

In [3]:
# Kết nối Milvus
def connect_to_milvus(uri="http://localhost:19530"):
    try:
        connections.connect(uri=uri)
        print("[INFO] Connected to Milvus.")
        print("list_collections: ", utility.list_collections())
    except MilvusException as e:
        print(f"[ERROR] Error connecting to Milvus: {e}")
        raise e

# Hàm kiểm tra trạng thái collection
def check_collection_status(col_name: str):
    try:
        if utility.has_collection(col_name):
            collection = Collection(name=col_name)
            print(f"[INFO] Collection '{col_name}' exists. Total entities: {collection.num_entities}")
        else:
            print(f"[INFO] Collection '{col_name}' does not exist.")
    except MilvusException as e:
        print(f"[ERROR] Error checking collection '{col_name}': {e}")
        raise e
# Main logic
if __name__ == "__main__":
    connect_to_milvus()

    # Tên collection
    col_name = "hybrid_demo"

    # Kiểm tra trạng thái collection
    check_collection_status(col_name)
    check_collection_status(col_name="base_qa")

[INFO] Connected to Milvus.
list_collections:  ['base_qa', 'raptor', 'hybrid_demo']
[INFO] Collection 'hybrid_demo' exists. Total entities: 27
[INFO] Collection 'base_qa' exists. Total entities: 2


In [ ]:
print("list_collections: ", utility.list_collections())
Collection(name=drop_collection_name).drop()
print("list_collections: ", utility.list_collections())

In [4]:
from milvus_db import MilvusDB, MilvusQADB# step4_vector_store
from utils import convert_df_to_documents
from model_config import VietnameseEmbeddings
import os
import pandas as pd
def load_final_df():
    final_df_path = r"D:\DATN\QA_System\data_analyze\finaldf.pkl"
    if os.path.exists(final_df_path):
        final_df = pd.read_pickle(final_df_path)
        print(f"Đã load final_df từ file: {final_df_path}")
        return final_df
    else:
        print("File final_df chưa tồn tại.")
        return None

final_df = load_final_df()
documents = convert_df_to_documents(final_df)
corpus=[doc.page_content for doc in documents]

c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Đã load final_df từ file: D:\DATN\QA_System\data_analyze\finaldf.pkl
Processed 1198 out of 1198 documents.


c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
milvus_db = MilvusDB("raptor", corpus=corpus)

Initializing Vietnamese embedding model: keepitreal/vietnamese-sbert


c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
milvus_db.create_collection()
milvus_db.insert_documents(documents)

In [6]:
milvus_db.load_collection()

In [7]:
relevant_docs = milvus_db.perform_retrieval(query="giám đốc đại học là ai", top_k=5)

In [8]:
relevant_docs

[Document(metadata={'doc_id': '454859381008765521', 'source': "{'source': 'Hội đồng Đại học Bách khoa Hà Nội.txt', 'id': '7'}", 'page': -1, 'level': 0}, page_content='Giám sát việc thực hiện quyết định của hội đồng trường, việc tuân thủ pháp luật, thực hiện quy chế dân chủ trong hoạt động của trường đại học và trách nhiệm giải trình của hiệu trưởng trường đại học; giám sát việc quản lý, sử dụng tài chính, tài sản của trường đại học; báo cáo hằng năm trước hội nghị toàn thể của trường đại học về kết quả giám sát và kết quả hoạt động của hội đồng trường;'),
 Document(metadata={'doc_id': '454859381008766241', 'source': "{'source': 'Trung tâm Nghiên cứu Quốc tế về Trí tuệ nhân tạo.txt', 'id': '0'}", 'page': -1, 'level': 0}, page_content='Giới\xa0thiệu chung:\nTên tiếng Anh: the International Research Center for Artificial Intelligence (BK.AI)\nGiám đốc khoa học:\xa0GS. Hồ Tú Bảo\nGiám đốc điều hành: \xa0TS. Nguyễn Phi Lê\nPhó giám đốc điều hành:\xa0TS. Đinh Viết Sang\nĐịa chỉ: Tầng 10, Nhà

In [ ]:
relevant_docs.metadata.get("source")

In [25]:
test = [doc.page_content + "- Source: " + doc.metadata.get("source") for doc in documents]

In [26]:
test

['QUY ĐỊNH Xét cấp học bổng khuyến khích học tập tại Đại học Bách khoa Hà Nội /2023 Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng 1. Văn bản này quy đị nh về nguyên t ắc, tiêu chu ẩn xét duyệt và cấp học bổng cho sinh viên đại học hình thức đào tạo chính- Source: 20230710 1. QĐ Học bổng KKHT 2023.pdf',
 'và cấp học bổng cho sinh viên đại học hình thức đào tạo chính quy tại Đại học Bách khoa Hà Nội (sau đây viết tắt là ĐHBK Hà Nội), căn cứ các quy định liên quan tại Nghị định số 84/2020/NĐ -CP ngày 17 /7/2020 của Chính phủ và Quy chế Quản lý tài chính của- Source: 20230710 1. QĐ Học bổng KKHT 2023.pdf',
 'ngày 17 /7/2020 của Chính phủ và Quy chế Quản lý tài chính của Đại học Bách khoa Hà Nội được ban hành theo Nghị quyết số 20/NQ-ĐHBK ngày 16/3/2023 của Hội đồng Đại học , Đại học Bách khoa Hà Nội . 2. Quy định này áp dụng đối với sinh viên đại học hình- Source: 20230710 1. QĐ Học bổng KKHT 2023.pdf',
 'Hà Nội . 2. Quy định này áp dụng đối với sinh viên đại học hình thức đào tạo chính 

In [13]:
corpus

['QUY ĐỊNH Xét cấp học bổng khuyến khích học tập tại Đại học Bách khoa Hà Nội /2023 Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng 1. Văn bản này quy đị nh về nguyên t ắc, tiêu chu ẩn xét duyệt và cấp học bổng cho sinh viên đại học hình thức đào tạo chính',
 'và cấp học bổng cho sinh viên đại học hình thức đào tạo chính quy tại Đại học Bách khoa Hà Nội (sau đây viết tắt là ĐHBK Hà Nội), căn cứ các quy định liên quan tại Nghị định số 84/2020/NĐ -CP ngày 17 /7/2020 của Chính phủ và Quy chế Quản lý tài chính của',
 'ngày 17 /7/2020 của Chính phủ và Quy chế Quản lý tài chính của Đại học Bách khoa Hà Nội được ban hành theo Nghị quyết số 20/NQ-ĐHBK ngày 16/3/2023 của Hội đồng Đại học , Đại học Bách khoa Hà Nội . 2. Quy định này áp dụng đối với sinh viên đại học hình',
 'Hà Nội . 2. Quy định này áp dụng đối với sinh viên đại học hình thức đào tạo chính quy văn bằng thứ nhất của ĐHBK Hà Nội . Điều 2. Các quy định chung 1. Học bổng khuyến khích học tập (KKHT) cấp cho các sinh viên được lựa chọ

In [ ]:
milvus_db.insert_documents

In [35]:
from langchain.schema import Document

final_df_path = r"D:\DATN\QA_System\data_analyze\finaldf.pkl"
if os.path.exists(final_df_path):
    final_df = pd.read_pickle(final_df_path)
final_df_path = r"D:\DATN\QA_System\data_analyze\finaldf0.pkl"
if os.path.exists(final_df_path):
    final_df0 = pd.read_pickle(final_df_path) 

def convert_df_to_documents(final_df):
    documents = []
    for _, row in final_df.iterrows():
        # Handle metadata - convert to dict if it's a string
        metadata = row["metadata"]
        if isinstance(metadata, str):
            source = metadata
        else:
            source = metadata.get("source", "unknown") if isinstance(metadata, dict) else "unknown"

        # Create Document with properly handled metadata
        doc = Document(
            page_content=row["text"],
            metadata={
                "source": source,
                "level": row["level"]
            }
        )
        documents.append(doc)

    # Process documents
    processed_documents = []
    for doc in documents:
        # Handle source field
        if isinstance(doc.metadata["source"], list):
            doc.metadata["source"] = " ".join(doc.metadata["source"]) if doc.metadata["source"] else "..."
        elif doc.metadata["source"] is None:
            doc.metadata["source"] = "..."

        # Skip empty content
        if not doc.page_content.strip():
            print(f"Warning: Page content is empty for document: {doc.metadata['source']}")
            continue

        processed_documents.append(doc)

    print(f"Processed {len(processed_documents)} out of {len(documents)} documents.")
    return processed_documents


In [36]:
documents = convert_df_to_documents(final_df)
documents0 = convert_df_to_documents(final_df0)

Processed 1198 out of 1198 documents.
Processed 981 out of 981 documents.


In [34]:
documents

[Document(metadata={'source': "{'source': 'Ban giám đốc Đại học.txt', 'id': '0'}", 'level': 0}, page_content='Giám đốc Đại học:\xa0PGS.TS. Huỳnh Quyết Thắng\nEmail: thang.huynhquyet@hust.edu.vn\xa0\nPhó Giám đốc:\xa0PGS.TS. Nguyễn Phong Điền\nEmail: dien.nguyenphong@hust.edu.vn\xa0\nPhó Giám đốc: PGS.TS. Huỳnh Đăng Chính\nEmail: chinh.huynhdang@hust.edu.vn\xa0\nPhó Giám đốc: PGS.TS. Trần Ngọc Khiêm\nEmail: khiem.tranngoc@hust.edu.vn'),
 Document(metadata={'source': "{'id': ['0'], 'source': ['Ban giám đốc Đại học.txt']}", 'level': 1}, page_content='Cụm văn bản này cung cấp thông tin liên hệ của các lãnh đạo cấp cao của Đại học Bách Khoa Hà Nội, bao gồm Giám đốc và ba Phó Giám đốc.  Cụ thể,  đó là tên, chức danh và địa chỉ email của PGS.TS. Huỳnh Quyết Thắng (Giám đốc), PGS.TS. Nguyễn Phong Điền, PGS.TS. Huỳnh Đăng Chính và PGS.TS. Trần Ngọc Khiêm (ba Phó Giám đốc).\n'),
 Document(metadata={'source': "{'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '0'}", 'level': 

In [37]:
documents0

[Document(metadata={'source': "{'source': 'Ban giám đốc Đại học.txt', 'id': '0'}", 'level': 0}, page_content='Giám đốc Đại học:\xa0PGS.TS. Huỳnh Quyết Thắng\nEmail: thang.huynhquyet@hust.edu.vn\xa0\nPhó Giám đốc:\xa0PGS.TS. Nguyễn Phong Điền\nEmail: dien.nguyenphong@hust.edu.vn\xa0\nPhó Giám đốc: PGS.TS. Huỳnh Đăng Chính\nEmail: chinh.huynhdang@hust.edu.vn\xa0\nPhó Giám đốc: PGS.TS. Trần Ngọc Khiêm\nEmail: khiem.tranngoc@hust.edu.vn'),
 Document(metadata={'source': "{'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '0'}", 'level': 0}, page_content='CHIẾN LƯỢC PHÁT TRIỂN\nTÓM LƯỢC CHIẾN LƯỢC\xa0PHÁT\xa0TRIỂN ĐẠI HỌC BÁCH KHOA HÀ\xa0NỘI\xa0\nGIAI ĐOẠN 2017-2025\nI. MỤC TIÊU\xa0\nPhát triển thành một đại học nghiên cứu đa lĩnh vực với nòng cốt là kỹ thuật và công nghệ, trong đó các đơn vị chuyên môn được tổ chức thành một số trường và khoa; viện và trung tâm nghiên cứu trực thuộc.\xa0\nXây dựng hình mẫu thành công, phát triển bền vững của một đại học tự chủ toàn diện 

In [48]:
documents0[0].metadata.get("source")

"{'source': 'Ban giám đốc Đại học.txt', 'id': '0'}"

In [53]:
doc_context = "\n\n".join([doc.page_content + doc.metadata.get("source") for doc in relevant_docs])

In [52]:
doc_context

"Giám sát việc thực hiện quyết định của hội đồng trường, việc tuân thủ pháp luật, thực hiện quy chế dân chủ trong hoạt động của trường đại học và trách nhiệm giải trình của hiệu trưởng trường đại học; giám sát việc quản lý, sử dụng tài chính, tài sản của trường đại học; báo cáo hằng năm trước hội nghị toàn thể của trường đại học về kết quả giám sát và kết quả hoạt động của hội đồng trường;{'source': 'Hội đồng Đại học Bách khoa Hà Nội.txt', 'id': '7'}\n\nGiới\xa0thiệu chung:\nTên tiếng Anh: the International Research Center for Artificial Intelligence (BK.AI)\nGiám đốc khoa học:\xa0GS. Hồ Tú Bảo\nGiám đốc điều hành: \xa0TS. Nguyễn Phi Lê\nPhó giám đốc điều hành:\xa0TS. Đinh Viết Sang\nĐịa chỉ: Tầng 10, Nhà B1, Đại học Bách khoa Hà Nội\nEmail:\xa0bkai@soict.hust.edu.vn\nWebsite:\xa0bkai.ai{'source': 'Trung tâm Nghiên cứu Quốc tế về Trí tuệ nhân tạo.txt', 'id': '0'}\n\nCụm văn bản mô tả về hợp tác quốc tế của một trung tâm (có lẽ là Trung tâm thuộc Đại học Bách Khoa Hà Nội) trong lĩnh vực